### 2．학습 데이터 준비하기

In [1]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn 라이브러리 임포트
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Pandas 라이브러리 임포트
import pandas as pd

In [ ]:
# 와인 데이터 읽어 들이기
wine = load_wine()
wine

In [ ]:
# 데이터프레임에 담긴 설명변수 출력
pd.DataFrame(wine.data, columns=wine.feature_names)

In [ ]:
# 목적변수 데이터 출력
wine.target

In [ ]:
# 설명변수와 목적변수를 변수에 대입
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [ ]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)

# 데이터 건수 확인
print(len(train_X))
print(len(test_X))

### 3．텐서 생성

In [ ]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

In [ ]:
# 설명변수와 목적변수의 텐서를 합침
train = TensorDataset(train_X, train_Y)

# 텐서의 첫 번째 데이터 내용 확인
print(train[0])

# 미니배치로 분할
train_loader = DataLoader(train, batch_size=16, shuffle=True)

### 4.ネットワークの作成

In [ ]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)

    # 인스턴스 생성
model = Net()

### 5．모형 학습

In [ ]:
# 오차함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    # 분할해 둔 데이터를 꺼내옴
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data[0]
    # 50회 반복마다 누적오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

In [ ]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)
# 출력이 0 혹은 1이 되게 함
result = torch.max(model(test_x).data, 1)[1]
# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy